In [5]:
import pandas as pd
import hashlib

In [6]:
data_path = "../../../database"

df = pd.read_csv(f"{data_path}/silver/arquivo_anonimizado_v2.csv")

In [8]:
colunas_aluno = [
    "sk_d_aluno",
    "nome_completo",
    "matricula_dre",
    "sexo",
    "idade",
    "data_nascimento",
    "periodo_ingresso_curso_atual",
    "periodo_ingresso_ufrj",
    "forma_ingresso",
    "modalidade_cota",
    "nota_enem",
]

d_aluno = df[colunas_aluno[1:]]
d_aluno = d_aluno.drop_duplicates().reset_index(drop=True)

d_aluno["sk_d_aluno"] = d_aluno.apply(
    lambda x: hashlib.sha256(str(x["matricula_dre"]).encode()).hexdigest(), axis=1
)
d_aluno = d_aluno.reindex(columns=colunas_aluno)

In [9]:
d_aluno.to_csv(f"{data_path}/gold/d_aluno.csv", index=False)

In [ ]:
def processar_fonte_GasOcidente(caminho, coluna_categoria):
    caminho_completo = f"{CAMINHO_DATALAKE}/{caminho}"

    dataframe = spark.read.format("delta").load(caminho_completo)

    # Criando a nova coluna "DS_PONTO_CATEGORIA"
    df = dataframe.withColumn("DS_PONTO_CATEGORIA", lit(coluna_categoria))
    df = df.withColumn("DS_TRANSPORTADORA", lit("GasOcidente do Mato Grosso - GOM"))
    df = df.withColumnRenamed("data", "DT_DATA")
    df = df.withColumnRenamed("medidor", "DS_PONTO_TRANSPORTE")

    return df


def filtrar_tipo_volume(dataframe, tipo, coluna_volume):
    dataframe = dataframe.filter(col("Tipo") == tipo).withColumnRenamed(
        "volume", coluna_volume
    )

    return dataframe.select(
        *[
            "DS_PONTO_TRANSPORTE",
            "DT_DATA",
            "DS_PONTO_CATEGORIA",
            "DS_TRANSPORTADORA",
            coluna_volume,
        ]
    )